### IBM Data Science Capstone Project
#### The Battle of Neighborhoods

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json 
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

print('Libraries imported.')

Libraries imported.


In [2]:
URL = "https://www.zip-codes.com/zip-code-radius-finder.asp?zipMilesLow=0&zipMilesHigh=25&zip1=03869&srch=Do+Radius+Search"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
def tableDataText(table):       
    rows = []
    trs = table.find_all('tr')
    headerow = [td.get_text(strip=True) for td in trs[0].find_all('th')] # header row
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append([td.get_text(strip=True) for td in tr.find_all('td')]) # data row
    return rows

In [81]:
table = soup.find("div", {"id" : "tableview"})
list_table = tableDataText(table)
df = pd.DataFrame(list_table[1:], columns=list_table[0])
df = df.drop('#', axis = 1)
df = df.drop_duplicates(subset = ['City'])
df = df[:-1]
df.head()

,Zip,City,County,St,Country,Distance,
0,03869,ROLLINSFORD,STRAFFORD,NH,US,0.00 miles,
1,03822,DOVER,STRAFFORD,NH,US,2.26 miles,
3,03878,SOMERSWORTH,STRAFFORD,NH,US,3.20 miles,
5,03908,SOUTH BERWICK,YORK,ME,US,5.29 miles,
6,03823,MADBURY,STRAFFORD,NH,US,5.74 miles,


In [93]:
lat = []
lon = []

geolocator = Nominatim(user_agent='geoapiExercises')

for town, state in zip(df['City'], df['St']):
    tmp = town + " " + state
    lat.append(geolocator.geocode(tmp).raw['lat'])
    lon.append(geolocator.geocode(tmp).raw['lon'])
    
df['Latitude'] = lat
df['Longitude'] = lon
df.head()

,Zip,City,County,St,Country,Distance,,Latitude,Longitude
0,03869,ROLLINSFORD,STRAFFORD,NH,US,0.00 miles,,43.236199,-70.820618
1,03822,DOVER,STRAFFORD,NH,US,2.26 miles,,43.198057,-70.8739158
3,03878,SOMERSWORTH,STRAFFORD,NH,US,3.20 miles,,43.2625585,-70.8645872
5,03908,SOUTH BERWICK,YORK,ME,US,5.29 miles,,43.2344819,-70.8089397
6,03823,MADBURY,STRAFFORD,NH,US,5.74 miles,,43.16963,-70.924301


In [112]:
rollinsford = df[df['City'] == 'Rollinsford'.upper()]
map_rollinsford = folium.Map(location = [rollinsford['Latitude'], rollinsford['Longitude']], zoom_start = 9)

for lat, lng, city, state in zip(df['Latitude'], df['Longitude'], df['City'], df['St']):
    label = '{}, {}'.format(city, state)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rollinsford)  

map_rollinsford